Clusentric distance

In [2]:
import tree.ctutils as ctu

import matplotlib.pyplot as plt
import pickle
import numpy as np
import load

In [29]:
wdir = './05427/'
mpgs = pickle.load(open(wdir + "main_prgs_5_10_0.5_0.5_0.5_37_0.01_filtered_.pickle", 'rb'))

cluster center 

In [30]:
import tree.halomodule as hmo

def get_main_halo(halo):
    return halo.data[np.argmax(halo.data["np"])]

In [31]:
nout = 187
info = load.info.Info(base=wdir, nout=nout)

hh = hmo.Halo(nout=nout, base=wdir)
main_cluster = get_main_halo(hh)
pos = np.array([main_cluster.x, main_cluster.y, main_cluster.z]) * info.pboxsize
#print(pos)

Distance from the cluster center

In [32]:
nout_ini = 54
nout_fi = 187

In [33]:
tt = pickle.load(open(wdir + 'halo/Trees/extended_tree.pickle', "rb"))
tt_now = tt.data[tt.data["nout"] == nout_fi]
cluster = tt_now[np.argmax(tt_now["np"])]
cluster_tree = ctu.extract_main_tree(tt.data, idx=cluster["id"])

In [34]:
ttg = pickle.load(open(wdir + 'GalaxyMaker/Trees/extended_tree.pickle', "rb"))
ttg_now = ttg.data[ttg.data["nout"] == nout_fi]
clusterg = ttg_now[np.argmax(ttg_now["np"])]
cluster_treeg = ctu.extract_main_tree(ttg.data, idx=clusterg["id"])

In [35]:
info_list = []
for i, nout in enumerate(range(nout_ini, nout_fi+1)):
    info = load.info.Info(base=wdir, nout=nout)
    info_list.append(info)

In [36]:
pbox_arr = np.array([info.pboxsize for info in info_list])
nout_arr = np.array([info.nout for info in info_list])
aexp_arr = np.array([info.aexp for info in info_list])

info_arr = np.rec.fromarrays((pbox_arr[::-1], nout_arr[::-1], aexp_arr[::-1]),
                               dtype=[("pbox", "<f8"),
                                      ("nout", "<i4"),
                                      ("aexp", "<f8")])



In [37]:
cluster_atree = ctu.extract_a_tree(tt.data, idx_last=cluster["id"])

1. lambda / d_lambda Vs clustocentric distance

----------------------------

In [40]:
dd_all=[]
lam_all=[]
color=[]
cluster_size =[]
for i in range(len(info_arr)):
    ClusterRvir = cluster_tree["rvir"] / 1e3 # * info_arr["pbox"]
    cx = cluster_treeg["x"][i]# in Mpc/h
    cy = cluster_treeg["y"][i]# 
    cz = cluster_treeg["z"][i]# 
    
    #xf = [ gal.data["xc"][i] for gal in mpgs]
    for j, gal in enumerate(mpgs):
        try:
            # galaxy position in Mpc unit.
            xf = gal.data["xc"][i] / info_arr["aexp"][i] * info.H0 * 1e-2
            yf = gal.data["yc"][i] / info_arr["aexp"][i] * info.H0 * 1e-2
            zf = gal.data["zc"][i] / info_arr["aexp"][i] * info.H0 * 1e-2

            dd = np.sqrt(np.square(xf - cx) +
                         np.square(yf - cy) + 
                         np.square(zf - cz))
            if dd > 20:
                continue
            dd_all.append(dd)
            lam_all.append(gal.data["lambda_r"][i])
            color.append(j)
            cluster_size.append(ClusterRvir)
            #ax.scatter(dd, gal.data["lambda_r"][i],
            #           c=i, cmap="jet")
        except:
            pass

In [41]:
# cluster infall time
gals = []
for j, gal in enumerate(mpgs):
    dd_all=[]
    lam_all=[]
    color=[]
    cluster_size =[]
    for i in range(len(info_arr)):
        ClusterRvir = cluster_tree["rvir"][i] / 1e3 # * info_arr["pbox"]
        cx = cluster_treeg["x"][i]# in Mpc/h
        cy = cluster_treeg["y"][i]# 
        cz = cluster_treeg["z"][i]# 

        try:
            # galaxy position in Mpc unit.
            xf = gal.data["xc"][i] / info_arr["aexp"][i] * info.H0 * 1e-2
            yf = gal.data["yc"][i] / info_arr["aexp"][i] * info.H0 * 1e-2
            zf = gal.data["zc"][i] / info_arr["aexp"][i] * info.H0 * 1e-2

            dd = np.sqrt(np.square(xf - cx) +
                         np.square(yf - cy) + 
                         np.square(zf - cz))
            if dd > 20:
                continue

            dd_all.append(dd)
            lam_all.append(gal.data["lambda_r"][i])
            color.append(j)
            cluster_size.append(ClusterRvir)
            #ax.scatter(dd, gal.data["lambda_r"][i],
            #           c=i, cmap="jet")
        except:
            pass
    
    gals.append((dd_all, lam_all, color, cluster_size))

In [42]:
color = np.array(color)
dd_all = np.array(dd_all)
lam_all = np.array(lam_all)
cluster_size = np.array(cluster_size)

In [43]:
t_inf = []
for gal in gals:
    dist = gal[0]
    csize = gal[3]
    t_inf.append(np.argmin(3*np.array(csize) > np.array(dist)))
    
t_inf = nout_fi - np.array(t_inf)

In [44]:
print(t_inf)

[187 124 153 108 165 124 148 127 136 173 149  91  67  97 104 115 107  78
 114  76 124 132 149 187 118 146 141 181 148 141 145  84 120 125 150 159
 133 154 155 146 106 160 154 136 146 125 156 146 124 151 147 128 153 121
 151 147 124 157 108  77 148 187 130 129  94 125]


In [45]:
lambda_fi = np.array([gal.data["lambda_r"][0] for gal in mpgs])
lambda_ini = np.array([gal.data["lambda_r"][-1] for gal in mpgs])
delta_lambda = lambda_fi - lambda_ini

fig, ax = plt.subplots(1,2)
ax[0].scatter(t_inf, lambda_fi)
ax[0].set_xlabel("t_infall")
ax[0].set_ylabel("Lambda at z=0")
ax[1].scatter(t_inf, delta_lambda)
ax[1].set_xlabel("t_infall")
ax[1].set_ylabel("Total Delta Lambda")
plt.show()

In [ ]:
fig, ax = plt.subplots()
for i in range(len(mpgs)):
    ind = np.where(color == i)[0]
    


In [209]:
fig, ax = plt.subplots()
for i in range(15):
    ind = np.where(color == i)[0]
    #ax.scatter(dd_all[ind],
    ax.plot(dd_all[ind],
               lam_all[ind])
    #inside_cluster = ClusterRvir < dd

In [210]:
plt.show()